---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the Jupyter Notebook FAQ course resource._

---

##### Assignment 4 - Predicting and understanding viewer engagement with educational videos 

With the accelerating popularity of online educational experiences, the role of online lectures and other educational video continues to increase in scope and importance. Open access educational repositories such as <a href="http://videolectures.net/">videolectures.net</a>, as well as Massive Open Online Courses (MOOCs) on platforms like Coursera, have made access to many thousands of lectures and tutorials an accessible option for millions of people around the world. Yet this impressive volume of content has also led to a challenge in how to find, filter, and match these videos with learners. This assignment gives you an example of how machine learning can be used to address part of that challenge.

## About the prediction problem

One critical property of a video is engagement: how interesting or "engaging" it is for viewers, so that they decide to keep watching. Engagement is critical for learning, whether the instruction is coming from a video or any other source. There are many ways to define engagement with video, but one common approach is to estimate it by measuring how much of the video a user watches. If the video is not interesting and does not engage a viewer, they will typically abandon it quickly, e.g. only watch 5 or 10% of the total. 

A first step towards providing the best-matching educational content is to understand which features of educational material make it engaging for learners in general. This is where predictive modeling can be applied, via supervised machine learning. For this assignment, your task is to predict how engaging an educational video is likely to be for viewers, based on a set of features extracted from the video's transcript, audio track, hosting site, and other sources.

We chose this prediction problem for several reasons:

* It combines a variety of features derived from a rich set of resources connected to the original data;
* The manageable dataset size means the dataset and supervised models for it can be easily explored on a wide variety of computing platforms;
* Predicting popularity or engagement for a media item, especially combined with understanding which features contribute to its success with viewers, is a fun problem but also a practical representative application of machine learning in a number of business and educational sectors.


## About the dataset

We extracted training and test datasets of educational video features from the VLE Dataset put together by researcher Sahan Bulathwela at University College London. 

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single educational video, and includes information about diverse properties of the video content as described further below. The target variable is `engagement` which was defined as True if the median percentage of the video watched across all viewers was at least 30%, and False otherwise.

Note: Any extra variables that may be included in the training set are simply for your interest if you want an additional source of data for visualization, or to enable unsupervised and semi-supervised approaches. However, they are not included in the test set and thus cannot be used for prediction. **Only the data already included in your Coursera directory can be used for training the model for this assignment.**

For this final assignment, you will bring together what you've learned across all four weeks of this course, by exploring different prediction models for this new dataset. In addition, we encourage you to apply what you've learned about model selection to do hyperparameter tuning using training/validation splits of the training data, to optimize the model and further increase its performance. In addition to a basic evaluation of model accuracy, we've also provided a utility function to visualize which features are most and least contributing to the overall model performance.

**File descriptions** 
    assets/train.csv - the training set (Use only this data for training your model!)
    assets/test.csv - the test set
<br>

**Data fields**

train.csv & test.csv:

    title_word_count - the number of words in the title of the video.
    
    document_entropy - a score indicating how varied the topics are covered in the video, based on the transcript. Videos with smaller entropy scores will tend to be more cohesive and more focused on a single topic.
    
    freshness - The number of days elapsed between 01/01/1970 and the lecture published date. Videos that are more recent will have higher freshness values.
    
    easiness - A text difficulty measure applied to the transcript. A lower score indicates more complex language used by the presenter.
    
    fraction_stopword_presence - A stopword is a very common word like 'the' or 'and'. This feature computes the fraction of all words that are stopwords in the video lecture transcript.
    
    speaker_speed - The average speaking rate in words per minute of the presenter in the video.
    
    silent_period_rate - The fraction of time in the lecture video that is silence (no speaking).
    
train.csv only:
    
    engagement - Target label for training. True if learners watched a substantial portion of the video (see description), or False otherwise.
    

## Evaluation

Your predictions will be given as the probability that the corresponding video will be engaging to learners.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model with an AUC (area under ROC curve) of at least 0.8 passes this assignment, and over 0.85 will receive full points.
___

For this assignment, create a function that trains a model to predict significant learner engagement with a video using `asset/train.csv`. Using this model, return a Pandas Series object of length 2309 with the data being the probability that each corresponding video from `readonly/test.csv` will be engaging (according to a model learned from the 'engagement' label in the training set), and the video index being in the `id` field.

Example:

    id
       9240    0.401958
       9241    0.105928
       9242    0.018572
                 ...
       9243    0.208567
       9244    0.818759
       9245    0.018528
             ...
       Name: engagement, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out and check your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. 

* Try to avoid global variables. If you have other functions besides engagement_model, you should move those functions inside the scope of engagement_model.

* Be sure to first check the pinned threads in Week 4's discussion forum if you run into a problem you can't figure out.

### Extensions

* If this prediction task motivates you to explore further, you can find more details here on the original VLE dataset and others related to video engagement: https://github.com/sahanbull/VLE-Dataset



In [2]:
import warnings
warnings.filterwarnings("ignore")

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(0)   # Do not change this value: required to be compatible with solutions generated by the autograder.

In [3]:
df=pd.read_csv('assets/train.csv')
X=df.drop(df.columns[[0,6,9]],axis=1)
y=df['engagement']
test=pd.read_csv('assets/test.csv')
test1=test.drop(df.columns[[0,6]],axis=1)
from sklearn.model_selection import train_test_split
X_train,X_test, y_train,y_test=train_test_split(X,y,random_state=0)


null=X_train.isnull()
null
for column in null.columns.values.tolist():
    print(column)
    print(null[column].value_counts())
    print("")
    #there are not missing value      
    
    
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

    #since there are possibility of overfit, we should adjust max_depth ,n_estimator, and max_depth
param_grid={'max_depth':[5,10,15],'max_features':[2,3],'n_estimators':[150]}
Grid=GridSearchCV(RandomForestClassifier(),param_grid,refit=True, verbose=3,scoring='roc_auc')
Grid.fit(X_train,y_train)
probs=Grid.predict_proba(test1)[:,1]
rec=pd.Series(probs,index=test.iloc[:,0])
rec

title_word_count
False    6929
Name: title_word_count, dtype: int64

document_entropy
False    6929
Name: document_entropy, dtype: int64

freshness
False    6929
Name: freshness, dtype: int64

easiness
False    6929
Name: easiness, dtype: int64

fraction_stopword_presence
False    6929
Name: fraction_stopword_presence, dtype: int64

speaker_speed
False    6929
Name: speaker_speed, dtype: int64

silent_period_rate
False    6929
Name: silent_period_rate, dtype: int64

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV 1/5] END max_depth=5, max_features=2, n_estimators=150;, score=0.875 total time=   0.8s
[CV 2/5] END max_depth=5, max_features=2, n_estimators=150;, score=0.891 total time=   0.8s
[CV 3/5] END max_depth=5, max_features=2, n_estimators=150;, score=0.875 total time=   0.8s
[CV 4/5] END max_depth=5, max_features=2, n_estimators=150;, score=0.886 total time=   0.8s
[CV 5/5] END max_depth=5, max_features=2, n_estimators=150;, score=0.878 total time=   0.8s
[CV 1/5] 

id
9240     0.015166
9241     0.039259
9242     0.068748
9243     0.883992
9244     0.047279
           ...   
11544    0.016809
11545    0.006192
11546    0.011961
11547    0.826351
11548    0.007522
Length: 2309, dtype: float64

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
for n in range(1,10):
    KNN=KNeighborsClassifier(n_neighbors=n)
    KNN.fit(X_train_scaled,y_train)
    print('for n={0}, the KNN train score is {1}'.format(n,KNN.score(X_train_scaled,y_train)))
    print('for n={0}, the KNN test score is {1}'.format(n,KNN.score(X_test_scaled,y_test)))
KNN.score(X_train_scaled,y_train)
#n=6 is the best

In [ ]:
#model evaluation
from sklearn.metrics import roc_auc_score

KNN6=KNeighborsClassifier(n_neighbors=6)
KNN6.fit(X_train_scaled,y_train)

print("Train set Accuracy: ", roc_auc_score(y_train, KNN6.predict(X_train_scaled)))
print("Test set Accuracy: ", roc_auc_score(y_test, KNN6.predict(X_test_scaled)))

In [ ]:
#model evaluation on different K value
from sklearn import metrics
Ks = 10
mean_acc = np.zeros((Ks-1))
std_acc = np.zeros((Ks-1))

for n in range(1,Ks):
    
    #Train Model and Predict  
    neigh = KNeighborsClassifier(n_neighbors = n).fit(X_train_scaled,y_train)
    yhat=neigh.predict(X_test_scaled)
    mean_acc[n-1] = metrics.accuracy_score(y_test, yhat)

    
    std_acc[n-1]=np.std(yhat==y_test)/np.sqrt(yhat.shape[0])

mean_acc

In [ ]:
#test with naive bayes
from sklearn.naive_bayes import GaussianNB
nbclf = GaussianNB().fit(X_train_scaled, y_train)
print('Accuracy of GaussianNB classifier on training set: {:.2f}'
     .format(nbclf.score(X_train_scaled, y_train)))
print('Accuracy of GaussianNB classifier on test set: {:.2f}'
     .format(nbclf.score(X_test_scaled, y_test)))


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
xgb=GradientBoostingClassifier()
param_grid={'max_depth':[3,4,5],'learning_rate':[.01,0.1,0.2,0.3,0.5, 0.7, 0.9],'n_estimators':[20,50,100,250,500]}
clf_xgb=GridSearchCV(xgb,param_grid=param_grid,verbose=True, n_jobs=-1)
clf_xgb.fit(X_train,y_train)

print('model score:{}'.format('clf_xgb.best_score_'))
print('model parameter:{}'.format('clf_xgb.best_params_'))

In [ ]:
#logistic regression
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()
param_grid={'C':np.logspace(-4,4,20),'penalty':['l1','l2'],'max_iter':[100,200,500,1000],'solver':['liblinear']}

clf_lr=GridSearchCV(lr,param_grid=param_grid,verbose=True,n_jobs=-1).fit(X_train_scaled,y_train)
print('the best parameter {}'.format(clf_lr.best_params_))
print('the best score {}'.format(clf_lr.best_score_))
print(clf_lr.score(X_test_scaled,y_test))
lr_predictions=clf_lr.predict(X_test_scaled)
print(classification_report(y_test,lr_predictions))
#X_test_scaled,X_train_scaled for the training 

In [ ]:
# try SVM neural network 
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
SVM=SVC()

clf_svc=GridSearchCV(SVM, param_grid=[{'C': [0.01, 0.1, 1, 10,100,1000],
                                         'gamma': [0.1, 0.5, 1, 2,5,10],
                                         'kernel': ['rbf']}], verbose=True, n_jobs=-1)
best_clf_rf=clf_svc.fit(X_train_scaled,y_train)


In [ ]:
print('the best parameter :{}'.format(clf_svc.best_params_))
print('the best score :{}'.format(clf_svc.best_score_))
from sklearn.metrics import classification_report, confusion_matrix 

clf_svc_predictions=best_clf_rf.predict(X_test)
print(classification_report(y_test,clf_svc_predictions))

In [4]:
def engagement_model():
    rec = None
    
    # YOUR CODE HERE
    df=pd.read_csv('assets/train.csv')
    X=df.drop(df.columns[[0,6,9]],axis=1)
    y=df['engagement']
    test=pd.read_csv('assets/test.csv')
    test1=test.drop(df.columns[[0,6]],axis=1)
    from sklearn.model_selection import train_test_split
    X_train,X_test, y_train,y_test=train_test_split(X,y,random_state=0)


    null=X_train.isnull()
    null
    for column in null.columns.values.tolist():
        print(column)
        print(null[column].value_counts())
        print("")
    #there are not missing value      
    
    
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import GridSearchCV

    #since there are possibility of overfit, we should adjust max_depth ,n_estimator, and max_depth
    param_grid={'max_depth':[5,10,15],'max_features':[2,3],'n_estimators':[150]}
    Grid=GridSearchCV(RandomForestClassifier(),param_grid,refit=True, verbose=3,scoring='roc_auc')
    Grid.fit(X_train,y_train)
    probs=Grid.predict_proba(test1)[:,1]
    rec=pd.Series(probs,index=test.iloc[:,0])
   
    
    
    return rec
    raise NotImplementedError()

In [5]:
stu_ans = engagement_model()
assert isinstance(stu_ans, pd.Series), "Your function should return a pd.Series. "
assert len(stu_ans) == 2309, "Your series is of incorrect length: expected 2309 "
assert np.issubdtype(stu_ans.index.dtype, np.integer), "Your answer pd.Series should have an index of integer type representing video id."

title_word_count
False    6929
Name: title_word_count, dtype: int64

document_entropy
False    6929
Name: document_entropy, dtype: int64

freshness
False    6929
Name: freshness, dtype: int64

easiness
False    6929
Name: easiness, dtype: int64

fraction_stopword_presence
False    6929
Name: fraction_stopword_presence, dtype: int64

speaker_speed
False    6929
Name: speaker_speed, dtype: int64

silent_period_rate
False    6929
Name: silent_period_rate, dtype: int64

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV 1/5] END max_depth=5, max_features=2, n_estimators=150;, score=0.874 total time=   0.7s
[CV 2/5] END max_depth=5, max_features=2, n_estimators=150;, score=0.892 total time=   0.7s
[CV 3/5] END max_depth=5, max_features=2, n_estimators=150;, score=0.867 total time=   0.7s
[CV 4/5] END max_depth=5, max_features=2, n_estimators=150;, score=0.886 total time=   0.7s
[CV 5/5] END max_depth=5, max_features=2, n_estimators=150;, score=0.880 total time=   0.8s
[CV 1/5] 